In [2]:
# import beam module
import apache_beam as beam

p = beam.Pipeline()

evens = ( p 
         | beam.Create(['one','two','three']) 
         | beam.Filter(lambda x: x % 2 == 0).with_input_types(int) 
        )
  
p.run()

TypeCheckError: Type hint violation for '[2]: Filter(<lambda at 2152007247.py:8>)': requires <class 'int'> but got <class 'str'> for x
Full type hint:
IOTypeHints[inputs=((<class 'int'>,), {}), outputs=None]
File "/home/danhnguyen/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3490, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
File "/home/danhnguyen/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
File "/tmp/ipykernel_14277/2152007247.py", line 8, in <module>
    | beam.Filter(lambda x: x % 2 == 0).with_input_types(int)
File "/home/danhnguyen/.local/lib/python3.10/site-packages/apache_beam/transforms/ptransform.py", line 920, in with_input_types
    return WithTypeHints.with_input_types(
File "/home/danhnguyen/.local/lib/python3.10/site-packages/apache_beam/typehints/decorators.py", line 538, in with_input_types
    self._type_hints =

In [3]:
import apache_beam as beam

p = beam.Pipeline()

# @beam.typehints.with_output_types(int)
@beam.typehints.with_input_types(int)
class FilterEvensDoFn(beam.DoFn):
  def process(self, element):
    if element % 2 == 0:
      yield element

evens = ( p
         | beam.Create(['1','2','3'])
         | beam.ParDo(FilterEvensDoFn()) 
        )
  
p.run()

TypeCheckError: Type hint violation for '[3]: ParDo(FilterEvensDoFn)': requires <class 'int'> but got <class 'str'> for element
Full type hint:
IOTypeHints[inputs=((<class 'int'>,), {}), outputs=None]
File "/home/danhnguyen/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3308, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
File "/home/danhnguyen/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3490, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
File "/home/danhnguyen/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
File "/tmp/ipykernel_14277/3109295789.py", line 6, in <module>
    class FilterEvensDoFn(beam.DoFn):
File "/home/danhnguyen/.local/lib/python3.10/site-packages/apache_beam/typehints/decorators.py", line 785, in annotate_input_types
    th = getattr(f, '_type_hi

In [ ]:
import apache_beam as beam
import typing

class Employee(object):
  def __init__(self, id, name):
    self.id = id
    self.name = name

class EmployeeCoder(beam.coders.Coder):

  def encode(self, employee):
    return ('%s:%s' % (employee.id, employee.name)).encode('utf-8')

  def decode(self, s):
    return Employee(*s.decode('utf-8').split(':'))

  def is_deterministic(self):
    return True

beam.coders.registry.register_coder(Employee, EmployeeCoder)

def split_file(input):
  name, id, salary = input.split(',')
  return Employee(id, name), int(salary)

result = (
    p
	| beam.io.ReadFromText('data.txt')
    | beam.Map(split_file)
    | beam.CombinePerKey(sum).with_input_types(typing.Tuple[Employee, int])
	)
	
p.run()	